# Setup notebook
Don't forget to activate the GPU usage in the notebook parameter. Click on `Modify -> Notebook parameter`. Under the `Hardware acceleration` section, choose `GPU`

# Clone or pull Valuenet

In [ ]:
!git clone https://github.com/brunnurs/valuenet.git
# !git pull

# Setup and install dependencies


In [ ]:
%cd valuenet/
!pip install ipywidgets
!spacy download en_core_web_sm
!pip install -r requirements.txt

# Verify GPU

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi

# Train Model
Here also, we follow the [user manual from Valuenet](https://github.com/brunnurs/valuenet).

## Training data

The data that will be used for training is located in the files

    valuenet/data/dataset/train.json
    valuenet/data/dataset/dev.json

with dataset being either hack_zurich or spider.



### Option A: Train with Spider dataset

In [3]:
data_set = 'spider'

### Option B: Train with Custom Data only

Create your custom data and preprocess it following the steps of [preprocess_custom_data_hack_zurich_2021.ipynb](https://github.com/hack-with-admin-ch/HackZurich2021/blob/main/notebooks/preprocess_custom_data_hack_zurich_2021.ipynb). 

Then, set dataset to 'hack_zurich' (as the cell below). 

In [4]:
data_set = 'hack_zurich'

### Option C: Train with Spider Dataset and the Custom Data

In this case, merge your custom data and spider data together in one json file for train, one for dev and one for the tables:
Concatenate
- valuenet/data/hack_zurich/train.json with valuenet/data/spider/train.json
- valuenet/data/hack_zurich/dev.json with valuenet/data/spider/dev.json
- valuenet/data/hack_zurich/original/tables.json with valuenet/data/spider/original/tables.json

and save the merged files in hack_zurich valuenet/data/hack_zurich/train.json, valuenet/data/hack_zurich/dev.json and valuenet/data/hack_zurich/original/tables.json.

As fine-tuning was never experimented with the Valuenet codebase, it is the safest way to train a model which will perform better on custom data.

In [5]:
data_set = 'hack_zurich'

## Training procedure

**Hyperparameters**:
Here, we set a few parameters but do not hesitate to look at [training parameters](https://github.com/brunnurs/valuenet/blob/hackzurich/src/config.py) if you want to specify other settings.

Train from scratch (with exception of the transformer encoder, which is already pre-trained). Fine-tuning was never implemented with valuent, feel free to try !

In [6]:
exp_name = "train-01" # Name of the training (saved in wandb and folder name in experiments)
batch_size = 8 # make it smaller if you encounter memory limitations 
num_epochs = 5

Be sure you have uploaded the dev.json and train.json files to `valuenet/data/hack_zurich` before performing the following steps

In [ ]:
%run src/main.py --exp_name {exp_name} --data_set {data_set} --cuda --batch_size {batch_size} --num_epochs {num_epochs}

**Results and final model**:
After training, you will be able to find the ground truth data, output of the model, evaluation results and intermediary data in the folder `valuenet/experiments/{exp_name}_date_time`.
If the accuracy was better than the last best accuracy, the model is also saved in the folder as `"best_model.pt"`.